In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import numpy as np
from math import gcd
import random
import matplotlib.pyplot as plt

def controlled_modular_exponentiation(circuit, a, power, N, control_qubit, target_qubits):
    """
    Implements controlled modular exponentiation: |x⟩|0⟩ → |x⟩|(a^x) mod N⟩
    This is a simplified version that works for small numbers
    """
    # For small numbers, we can implement this directly
    # For larger numbers, we would need a more sophisticated implementation
    for i in range(len(target_qubits)):
        if (a ** power) % N & (1 << i):
            # Use a simple CNOT for the first bit
            if i == 0:
                circuit.cx(control_qubit, target_qubits[i])
            else:
                circuit.ccx(control_qubit, target_qubits[i-1], target_qubits[i])




In [ ]:
def qft(circuit, n):
    """
    Implements the Quantum Fourier Transform on n qubits
    """
    # Apply the QFT
    for i in range(n):
        circuit.h(i)
        for j in range(i + 1, n):
            circuit.cp(np.pi / (2 ** (j - i)), j, i)
    
    # Apply SWAP gates to reverse the order
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

In [ ]:
def find_period_of_a_quantum(N, a):
        # Number of qubits needed
    n_counting = int(np.ceil(np.log2(N)))
    n_work = int(np.ceil(np.log2(N)))
    
    # Create quantum circuit
    circuit = QuantumCircuit(n_counting + n_work, n_counting)
    
    # Initialize counting qubits in superposition
    for i in range(n_counting):
        circuit.h(i)
    
    # Apply controlled modular exponentiation
    for i in range(n_counting):
        controlled_modular_exponentiation(
            circuit,
            a,
            2**i,
            N,
            i,
            list(range(n_counting, n_counting + n_work))
        )
    
    # Apply inverse QFT
    qft(circuit, n_counting)
    
        # Draw the circuit diagram
    print(circuit)

    # Measure the counting qubits
    circuit.measure(range(n_counting), range(n_counting))
    
    # Run the circuit
    simulator = Aer.get_backend('qasm_simulator')
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print(f"Counts: {counts}")


    # Find the most common measurement
    measured_value = max(counts, key=counts.get)
    phase = int(measured_value, 2) / (2 ** n_counting)
    
    # Convert phase to fraction
    from fractions import Fraction
    frac = Fraction(phase).limit_denominator(N)
    period = frac.denominator

    return period

In [ ]:

def shor(N, a=None):
    """
    Implements Shor's algorithm to factorize N
    """
    if N % 2 == 0:
        return "Divisible by 2. No Quantum Circuit needed.", 2, N // 2
    
    if a is None:
        a = random.randint(2, N - 1)
        print(f"Randomly selected a: {a}")
    
    # Check if a and N are coprime
    if gcd(a, N) != 1:
        return f"Random guess a:{a} is not coprime with N:{N}, meaning a shares a non-trivial factor determined by Eucliean algorithm.", gcd(a, N), N // gcd(a, N)
    
    print(f"Random guess a:{a} is coprime with N:{N}, so we can proceed with Shor's algorithm.")

    period = find_period_of_a_quantum(N, a)
    
    # If period is odd, try again
    if period % 2 == 1:
        print(f"For a {a}, period {period} is odd, trying again...")
        return shor(N)
    
    # Calculate potential factors
    factor1 = gcd(a ** (period // 2) - 1, N)
    factor2 = gcd(a ** (period // 2) + 1, N)
    print(f"Quantum Factors of {N}: {factor1, factor2}")
    
    if factor1 == 1 or factor1 == N or factor2 == 1 or factor2 == N:
        print(f"Quantum Factors of {N}: {factor1, factor2} are trivial, trying again...")
        return shor(N)
    
    return factor1, factor2


In [ ]:
if __name__ == "__main__":
    # Test with a small number
    N = 9
    print(f"Factoring {N}...")
    factor1, factor2 = shor(N)
    print(f"Factors of {N}: {factor1, factor2}") 
    print(drawing)

Factoring 9...
Randomly selected a: 8
Random guess a:8 is coprime with N:9, so we can proceed with Shor's algorithm.
     ┌───┐     ┌───┐                                                       »
q_0: ┤ H ├──■──┤ H ├─■────────■─────────────■──────────────────────────────»
     ├───┤  │  └───┘ │P(π/2)  │       ┌───┐ │                              »
q_1: ┤ H ├──┼────■───■────────┼───────┤ H ├─┼────────■────────■────────────»
     ├───┤  │    │            │P(π/4) └───┘ │        │P(π/2)  │       ┌───┐»
q_2: ┤ H ├──┼────┼──────■─────■─────────────┼────────■────────┼───────┤ H ├»
     ├───┤  │    │      │                   │P(π/8)           │P(π/4) └───┘»
q_3: ┤ H ├──┼────┼──────┼────────■──────────■─────────────────■────────────»
     └───┘  │  ┌─┴─┐  ┌─┴─┐    ┌─┴─┐                                       »
q_4: ───────┼──┤ X ├──┤ X ├────┤ X ├───────────────────────────────────────»
            │  └───┘  └───┘    └───┘                                       »
q_5: ───────┼───────────────────────